In [1]:
import spotipy, pandas as pd, json
from spotipy.oauth2 import SpotifyOAuth

In [2]:
scope = 'playlist-modify-public playlist-modify-private'
red_url = 'http://localhost:8080'

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(redirect_uri=red_url,scope=scope))

In [3]:
songs = sp.playlist_tracks(playlist_id='https://open.spotify.com/playlist/37i9dQZEVXcD0rEV1cHpue?si=85e21e56fd774936', limit=100)

In [4]:
song_ids = []
i = 0
for song in songs['items']:
    song_ids.append(songs['items'][i]['track']['id'])
    i += 1

In [5]:
song_feats = sp.audio_features(tracks=song_ids)

In [6]:
aud_anal = []
aud_secs = []
j = 0
for song in song_ids:
    track = sp.audio_analysis(song_ids[j])['track']
    dur = track['duration']
    secs = len(sp.audio_analysis(song_ids[j])['sections'])
    aud_anal.append(track)
    aud_secs.append(60*secs/dur)
    j += 1

In [7]:
df_ids = pd.DataFrame(song_ids)
df_feats = pd.DataFrame(song_feats).drop(['speechiness', 'acousticness', 'instrumentalness', 'liveness', 'type', 'uri', 'track_href', 'analysis_url', 'duration_ms'], axis=1)
df_aud = pd.DataFrame(aud_anal)[['tempo_confidence','time_signature_confidence','key_confidence','mode_confidence']]
df_secs = pd.DataFrame({'secs/min': aud_secs})

In [8]:
df_tot = df_feats.join([df_aud, df_secs]).set_index('id')